In [21]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import torch
import torch.utils.data as D
import torch.nn as nn
import pandas as pd
import os
import albumentations as A
from torchvision import transforms as T
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import Unet, FPN
import time
import shutil
import magic
try:
    import tkinter as tk
    from tkinter import ttk
    from tkinter import filedialog
except ImportError:
    import Tkinter as tk
    import ttk
    import tkFileDialog as filedialog
#from numba import njit
print('Success')

Success


In [2]:
torch.cuda.is_available()

/home/gatupov/anaconda3/envs/ml/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [64]:
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)
def ppr(x, nx): # просто форматированный вывод переменной
    print(nx + ': ' + str(x))

In [5]:
WINDOW = 128
MIN_OVERLAP = 16
NEW_SIZE = 256
DATA_DIR = 'coco_dataset/'
DEVICE = 'cpu'

trfm = A.Compose([
    A.Resize(NEW_SIZE,NEW_SIZE),
])
as_tensor = T.Compose([
            T.ToTensor(),
            T.Normalize([0.625, 0.448, 0.688],
                        [0.131, 0.177, 0.101]),
        ])

In [6]:
DIR_OF_MODEL = 'CatPytorchModel'
name_model = 'base_model'

In [7]:
def get_model():
    ENCODER = 'se_resnext50_32x4d'
    ENCODER_WEIGHTS = 'imagenet'
    CLASSES_NUMBER = 1
    ACTIVATION = 'sigmoid'  # could be None for logits or 'softmax2d' for multicalss segmentation

    # create segmentation model with pretrained encoder
    model = smp.Unet(
        encoder_name=ENCODER,
        encoder_weights=ENCODER_WEIGHTS,
        classes=CLASSES_NUMBER,
        activation=ACTIVATION,
    )

    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

    return model

In [8]:
model = get_model()
model.load_state_dict(torch.load(DIR_OF_MODEL + '/' + name_model))
model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [9]:
BATCH_SIZE = 5

In [67]:
 #######################
@torch.no_grad()
def finding_of_cat(name_img, data_dir=DATA_DIR, window=WINDOW, min_overlap=MIN_OVERLAP,
                  batch_size=BATCH_SIZE, new_size=NEW_SIZE, make_grid=make_grid,trfm=trfm,
                  as_tensor=as_tensor,device=DEVICE):
    image = io.imread(data_dir + '/' + name_img)
    #ppr(image.shape,'image_shape')
    slices = make_grid([image.shape[0],image.shape[1]], window=window, \
                                    min_overlap=min_overlap)
    image_tensor_slices = []
    for slc in slices:
        x1,x2,y1,y2 = slc
        img_slice = image[x1:x2,y1:y2]
        image_tensor = as_tensor(trfm(image=img_slice)['image'])
        image_tensor_slices.append(image_tensor)
    number_of_slices = len(image_tensor_slices)

    test_loader = D.DataLoader(
        image_tensor_slices, batch_size=batch_size, shuffle=False, num_workers=0)

    cat_pixels = 0
    for image_input in test_loader:
        image_input = image_input.to(device)
        output = model(image_input).cpu().detach().numpy()
        for slc in output:
            cat_pixels += slc[0].sum()

    #ppr(cat_pixels,'cat_pixels') 
    cat_pixels_persentage = float(cat_pixels/(new_size*new_size*number_of_slices))
    return cat_pixels_persentage
    #616895.5000


In [18]:
# start_time = time.time()
# print(finding_of_cat('img_5'))
# print('Time:')
# print(time.time()-start_time)

In [19]:
# #visualisation:
# plt.axis('off')
# plt.imshow(image)
# plt.show()

In [68]:
root = tk.Tk()

style = ttk.Style(root)
style.theme_use("clam")
root.configure(bg=style.lookup('TFrame', 'background'))
foto_dir = './'
def c_open_dir_old():
    dirr = os.getcwd()
    global foto_dir
    foto_dir = filedialog.askdirectory(parent=root, initialdir=dirr)
    root.destroy()
ttk.Button(root, text="Open folder", command=c_open_dir_old).grid(row=2, column=0, padx=4, pady=4, sticky='ew')
root.mainloop()
print(foto_dir)
fotos = os.listdir(foto_dir)
#print(fotos)
#fotos_filtered = []
#print(fotos)
number_files = len(fotos)
file_counter = 0
print('Определение форматов... ', end='')
fotos_filtered = []
for f in fotos:
#     percent = round(file_counter/number_files,2)
#     print('Определение форматов: {}/{}, {} %'.format(file_counter, number_files, \
#                                            int(percent*100)), end='\r')
#     time.sleep(0.0005)
    path_to_file = foto_dir + '/' + f
    if not(os.path.isdir(path_to_file)):
        file_type = magic.from_file(path_to_file)
        if file_type.split()[0] == 'PNG' or file_type.split()[0] == 'JPEG':
            fotos_filtered.append(f)
    file_counter += 1
#print(fotos_filtered)
print('Done')

# changing of dirrectory and copy the files.:
old_directory = os.getcwd()
os.chdir(foto_dir)
dir_of_cats_foto = 'photos_with_cats'
if not os.path.exists(dir_of_cats_foto):
    os.mkdir(dir_of_cats_foto)

number_fotos = len(fotos_filtered)
foto_counter = 0
print()
if number_fotos*8/60 < 1:
    print('Оценочное время работы программы: менее минуты')
else:
    print(f'Оценочное время работы программы: {int(number_fotos*27/60)} min,\
     ({round(int(number_fotos*8/3600),1)} ч)')
start_time = time.time()
print()
for f in fotos_filtered:
    percent = round(foto_counter/number_fotos,2)
    print('Обработано фото: {}/{}, {} %'.format(foto_counter, number_fotos, \
                                           int(percent*100)), end='\r')
    #time.sleep(0.0002)
    #print(foto_dir + '/' + f)
    if finding_of_cat(f, data_dir=foto_dir) >= 0.05:
        shutil.copy(f, dir_of_cats_foto)
    foto_counter += 1
print()
print('Done!')
time.sleep(0.001)
print(f'Общее время работы: {time.time() - start_time} c')
os.chdir(old_directory)


#     for path_to_foto in foto_of_cats:
#         shutil.copy(path_to_foto,'photos_with_cats')

/home/gatupov/python/ML_project/test_dataset
Определение форматов... Done

Оценочное время работы программы: 1 min,      (0 ч)

Done!отано фото: 9/10, 90 %
Общее время работы: 267.64606404304504 c


In [70]:
#os.chdir('/home/gatupov/python/ML_project')

26.8